# Disease Analysis
Analyze data from the National Notifiable Diseases Surveillance System on [deaths by pneumonia and influenza](https://www.healthdata.gov/dataset/nndss-table-ii-invasive-pneumococcal-diseases-all-ages) in 122 cities.

In [59]:
import numpy as np
from scipy import stats
import cntk as C

In [60]:
dataPath = "source.csv"
data = np.genfromtxt(dataPath, delimiter = ",", dtype = str)

In [61]:
deaths = data[1:, 6]
print(deaths)

non_blanks = []
for i in range(len(deaths)):
    if deaths[i] != "":
        non_blanks.append(i)
        
deaths = deaths[non_blanks]
years = stats.zscore( (data[1:, 0])[non_blanks].astype(np.float32))
winter = np.asarray( [1.0 if (week < 7 or 47 < week) else 0.0 for week in data[1:, 1].astype(int)], dtype = np.float32 )
winter = winter[non_blanks]

['11' '11' '5' ..., '2' '12' '11']


In [62]:
def shuffle_arrays(*arrays):
    rand_state = np.random.get_state()
    for array in arrays:
        np.random.set_state(rand_state)
        np.random.shuffle(array)

shuffle_arrays(years, winter, deaths)

In [63]:
num_samples = len(years)
training_size = num_samples * 2 // 3;


In [64]:
num_features = 2
num_classes = 1 #Do I even need this, since this is a regression problem?
input_var = C.input_variable(num_features, np.float32)

In [65]:
parm_dict = {}
def linear_layer(input_var, num_output_classes):
    
    weights = C.parameter(shape = (input_var.shape[0], num_output_classes))
    biases = C.parameter(shape = (num_output_classes))
    
    return C.times(input_var, weights) + biases

In [66]:
z = linear_layer(input_var, num_classes)
label = C.input_variable((num_classes), np.float32)
loss = C.squared_error(z, label)
#eval_error = C.squared_error(z, label)

In [71]:
learning_rate = 0.5
minibatch_size = 25

lr_schedule = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
learner = C.sgd(z.parameters, lr_schedule)

trainer = C.Trainer(z, (loss), [learner])

In [72]:
data = np.hstack(years, winter)
training_features = data[:training_size]
training_labels = deaths[:training_size]



def print_progress(trainer):
    print(0)

TypeError: hstack() takes 1 positional argument but 2 were given